In [ ]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy import optimize

import sys
sys.path.append('.../bats-code')
from bat_functions import get_bat_accumulation

In [ ]:
def save_fig(save_folder, plot_title, fig=None):
    plot_name = plot_title.replace(' ', '-')
    file = os.path.join(save_folder, plot_name+'.png')
    if fig:
        fig.savefig(file, bbox_inches='tight', dpi=600)
        return
    
    plt.savefig(file, bbox_inches='tight', dpi=600)

In [ ]:
falloff_csv = '.../bats-data/bat-counting-error-falloff.csv'
orig_csv = '.../bats-data/bat_counting_error_quantification_with_counts_padded.csv'

In [ ]:
plots_save_folder = '.../bats-data/plots/falloff-validation'

In [ ]:
val_df = pd.read_csv(falloff_csv)
orig_df = pd.read_csv(orig_csv)

In [ ]:
val_df['clip-num'] = [n.split('-')[-6] for n in val_df['video-clip-name']]
val_df['camera-name'] = [n.split('-')[-8] for n in val_df['video-clip-name']]

orig_df['clip-num'] = [n.split('-')[-6] for n in orig_df['Video clip name']]
orig_df['camera-name'] = [n.split('-')[-8] for n in orig_df['Video clip name']]

In [ ]:
total_bat_values = np.sum(~np.isnan(val_df['total-bats']))
old_bat_values = np.sum(~np.isnan(val_df['total-bats-old-method']))
new_bat_values = np.sum(~np.isnan(val_df['total-bats-new-method']))

orig_total_bat_values = np.sum(~np.isnan(orig_df['total_bats']))
orig_old_bat_values = np.sum(~np.isnan(orig_df.loc[~np.isnan(orig_df['total_bats']), 'total_bats_old_method']))
orig_new_bat_values = np.sum(~np.isnan(orig_df.loc[~np.isnan(orig_df['total_bats']), 'total_bats_new_method']))

print(f'total bat values: {total_bat_values} '
     + f'old bat values: {old_bat_values} '
     + f'new bat values: {new_bat_values}')
print(f'total bat values: {orig_total_bat_values} '
     + f'old bat values: {orig_old_bat_values} '
     + f'new bat values: {orig_new_bat_values}')

In [ ]:
has_val = ~np.isnan(val_df['total-bats'])
total_bats = np.sum(val_df.loc[has_val, 'total-bats'])
counted_new = np.sum(val_df.loc[has_val, 'total-bats-new-method'])
counted_old = np.sum(val_df.loc[has_val, 'total-bats-old-method'])

orig_has_val = ~np.isnan(orig_df['total_bats'])
orig_total_bats = np.sum(orig_df.loc[orig_has_val, 'total_bats'])
orig_counted_new = np.sum(orig_df.loc[orig_has_val, 'total_bats_new_method'])
orig_counted_old = np.sum(orig_df.loc[orig_has_val, 'total_bats_old_method'])

In [ ]:
print(f'new: {counted_new/total_bats}, old: {counted_old/total_bats}')
print(f'orig: new: {orig_counted_new/orig_total_bats}, old: {orig_counted_old/orig_total_bats}')
print(f'overal: new {(counted_new + orig_counted_new) / (total_bats + orig_total_bats)}')

print(f'overal: old {(counted_old + orig_counted_old) / (total_bats + orig_total_bats)}')

### Max bats in validation clips

In [ ]:
np.nanmean((orig_df['total_bats'] / 15) + (val_df['total-bats'] / 7))

In [ ]:
orig_df.head()

### Do false positives cancel out?

In [ ]:
grouped = val_df.groupby(['date-folder','camera-name'])

ave_noise = []

for name, group in grouped:
    val_net_noise = (group.loc[has_val, 'number-false-detections-going(blue)']
             - group.loc[has_val, 'number-false-detections-coming(yellow)']
            )
    ave_noise.append(np.mean(val_net_noise))

plt.hist(ave_noise, bins=50)

In [ ]:
val_net_noise = (val_df.loc[has_val, 'number-false-detections-going(blue)']
             - val_df.loc[has_val, 'number-false-detections-coming(yellow)']
            )


plt.hist(val_net_noise.values, bins=50)
plt.xlabel('Net difference between false detections moving up vs. down')
plt.ylabel('Number of validation clips')
plot_title = 'noise canceling out'
save_fig(plots_save_folder, plot_title, fig=None)


plt.figure()
plt.hist((val_df.loc[has_val, 'number-false-detections-going(blue)']
 + val_df.loc[has_val, 'number-false-detections-coming(yellow)']
).values)

### Precision

In [ ]:
plt.hist((val_df.loc[has_val, 'total-bats']
 - val_df.loc[has_val, 'number-missed-detections-coming']
).values)

plt.figure()
plt.hist((val_df.loc[has_val, 'number-missed-detections-going']
 + val_df.loc[has_val, 'number-missed-detections-coming']
).values)

In [ ]:
val_df.columns

In [ ]:
# val_df[val_df['camera-name']=="MusolePath"]

In [ ]:
plt.figure(figsize=(20,20))
plt.scatter(val_df['darkness-mean'][val_df['new-method-fraction-total']>0], 
            val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0], 
             label='mean-darkness')

# plt.figure()

plt.scatter(orig_df['darkness_mean'], 
            orig_df['new_method_fraction_total'], 
             label='mean_darkness')

# plt.scatter(val_df['darkness-mean'][val_df['old-method-fraction-total']>0], 
#             val_df['old-method-fraction-total'][val_df['old-method-fraction-total']>0], 
#              label='mean-darkness', marker='x')

# # plt.figure()

# plt.scatter(orig_df['darkness_mean'][orig_df['old_method_fraction_total']>0], 
#             orig_df['old_method_fraction_total'][orig_df['old_method_fraction_total']>0], 
#              label='mean_darkness', marker='x')

In [ ]:
# orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0))].loc[np.isnan(orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'new_method_fraction_total'])]

In [ ]:
not_zero = orig_df['total_bats'] != 0
orig_df['old_method_fraction_total'] = np.nan
orig_df['new_method_fraction_total'] = np.nan
orig_df.loc[not_zero, 'old_method_fraction_total'] = orig_df['total_bats_old_method'][not_zero] / orig_df['total_bats'][not_zero]
orig_df.loc[not_zero, 'new_method_fraction_total'] = orig_df['total_bats_new_method'][not_zero] / orig_df['total_bats'][not_zero]

len(orig_x), len(orig_y), len(orig_weights)


In [ ]:
##### 
import matplotlib.pyplot as plt
import numpy as np

save = False

x_pruned = val_df['darkness-mean'][val_df['new-method-fraction-total']>0].values
y_pruned = val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0].values
weights_pruned = val_df['total-bats'][val_df['new-method-fraction-total']>0].values

orig_x_pruned = orig_df['darkness_mean'][orig_df['new_method_fraction_total']>0].values
orig_y_pruned = orig_df['new_method_fraction_total'][orig_df['new_method_fraction_total']>0].values
orig_weights_pruned = orig_df['total_bats'][orig_df['new_method_fraction_total']>0].values

x = val_df['darkness-mean'].values
y = val_df['new-method-fraction-total'].values
weights = val_df['total-bats'].values

orig_x = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'darkness_mean'].values
orig_y = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'new_method_fraction_total'].values
orig_weights = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'total_bats'].values

# x = np.hstack([x, orig_x])
# y = np.hstack([y, orig_y])
# weights = np.hstack([weights, orig_weights])

sigmas = 1 / weights

def piecewise_linear(x, x0, y0, k1, k2):
    return np.piecewise(x, [x < x0], [lambda x:k1*x + y0-k1*x0, lambda x:k2*x + y0-k2*x0])


xd = np.linspace(5, 120, 120)
# x_pruned = np.hstack([x_pruned, orig_x_pruned])
# y_pruned = np.hstack([y_pruned, orig_y_pruned])
# weights_pruned = np.hstack([weights_pruned, orig_weights_pruned])
plt.scatter(x_pruned, y_pruned, 
            s=weights_pruned, alpha=.4)
p , e = optimize.curve_fit(piecewise_linear, x, y, p0=[10, 1, 1, -1], sigma=sigmas)
plt.plot(xd, piecewise_linear(xd, *p))
plot_title = "Darkness detection accuracy falloff 10"
plt.title(plot_title)
plt.xlabel("Average pixel value of frame (blue channel)")
plt.ylabel("Detection Accuracy")
print(p)
perr = np.sqrt(np.diag(e))
print(perr)
if save:
    save_fig(plots_save_folder, plot_title, fig=None)

In [ ]:
p , e = optimize.curve_fit(piecewise_linear, x, y, p0=[10, 1, 1, 0.5], sigma=sigmas)


In [ ]:
save = False
x_pruned = val_df['darkness-mean'][val_df['new-method-fraction-total']>0].values
y_pruned = val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0].values
weights_pruned = val_df['total-bats'][val_df['new-method-fraction-total']>0].values

orig_x_pruned = orig_df['darkness_mean'][orig_df['new_method_fraction_total']>0].values
orig_y_pruned = orig_df['new_method_fraction_total'][orig_df['new_method_fraction_total']>0].values
orig_weights_pruned = orig_df['total_bats'][orig_df['new_method_fraction_total']>0].values

x = val_df['darkness-mean'].values
y = val_df['new-method-fraction-total'].values
weights = val_df['total-bats'].values

orig_x = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'darkness_mean'].values
orig_y = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'new_method_fraction_total'].values
orig_weights = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'total_bats'].values 

x = np.hstack([x, orig_x])
y = np.hstack([y, orig_y])
weights = np.hstack([weights, orig_weights])

sigmas = 1 / weights

def piecewise_linear(x, x0, y0, k1, k2):
    return np.piecewise(x, [x < x0], [lambda x:k1*x + y0-k1*x0, lambda x:k2*x + y0-k2*x0])


xd = np.linspace(0, 170, 170)
x_pruned = np.hstack([x_pruned, orig_x_pruned])
y_pruned = np.hstack([y_pruned, orig_y_pruned])
weights_pruned = np.hstack([weights_pruned, orig_weights_pruned])
# Raw
# plt.scatter(x, y, 
#             s=weights, alpha=.4, label='Falloff validation', c='k')

# Pruned
plt.scatter(x_pruned, y_pruned, 
            s=weights_pruned, alpha=.4, label='Falloff validation', c='k')

# Center points
# plt.scatter(x_pruned, y_pruned, 
#             s=1, alpha=.4, label='Falloff validation', c='w')

# Just original validation
# plt.scatter(orig_x_pruned, orig_y_pruned, 
#             s=orig_weights_pruned/2, alpha=.4, label='Original validation', c='b')

p , e = optimize.curve_fit(piecewise_linear, x, y, p0=[10, 1, 1, 1], sigma=sigmas)
plt.plot(xd, piecewise_linear(xd, *p), c='gray')
plt.margins(x=0)
plt.xticks(np.arange(0, 180, 20), np.arange(0, 180, 20))

# plt.legend()

plot_title = "Darkness detection accuracy all black all points"
# plt.title(plot_title)
plt.xlabel("Average pixel value of frame (blue channel)")
plt.ylabel("Detection Accuracy")

print(p)
if save:
    save_fig(plots_save_folder, plot_title, fig=None)

In [ ]:
print(f'function 1: y = {p[2]}x + {p[1] - p[2]*p[0]}')
print(f'function 2: y = {p[3]}x + {p[1] - p[3]*p[0]}')

In [ ]:
save = False
x_pruned = val_df['darkness-mean'][val_df['new-method-fraction-total']>0].values
y_pruned = val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0].values
weights_pruned = val_df['total-bats'][val_df['new-method-fraction-total']>0].values

orig_x_pruned = orig_df['darkness_mean'][orig_df['new_method_fraction_total']>0].values
orig_y_pruned = orig_df['new_method_fraction_total'][orig_df['new_method_fraction_total']>0].values
orig_weights_pruned = orig_df['total_bats'][orig_df['new_method_fraction_total']>0].values

x = val_df['darkness-mean'].values
y = val_df['new-method-fraction-total'].values
weights = val_df['total-bats'].values

orig_x = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'darkness_mean'].values
orig_y = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'new_method_fraction_total'].values
orig_weights = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'total_bats'].values 

x = np.hstack([x, orig_x])
y = np.hstack([y, orig_y])
weights = np.hstack([weights, orig_weights])

sigmas = 1 / weights

def piecewise_linear(x, x0, y0, k1, k2):
    return np.piecewise(x, [x < x0], [lambda x:k1*x + y0-k1*x0, lambda x:k2*x + y0-k2*x0])


xd = np.linspace(0, 170, 170)
x_pruned = np.hstack([x_pruned, orig_x_pruned])
y_pruned = np.hstack([y_pruned, orig_y_pruned])
weights_pruned = np.hstack([weights_pruned, orig_weights_pruned])
# Raw
# plt.scatter(x, y, 
#             s=weights, alpha=.4, label='Falloff validation', c='k')

# Pruned
plt.scatter(x, y, 
            s=weights, alpha=.4, label='Falloff validation', c='k')

# Center points
# plt.scatter(x_pruned, y_pruned, 
#             s=1, alpha=.4, label='Falloff validation', c='w')

# Just original validation
# plt.scatter(orig_x_pruned, orig_y_pruned, 
#             s=orig_weights_pruned/2, alpha=.4, label='Original validation', c='b')

p , e = optimize.curve_fit(piecewise_linear, x, y, p0=[10, 1, 1, 1], sigma=sigmas)
plt.plot(xd, piecewise_linear(xd, *p), c='gray')
plt.margins(x=0)
plt.xticks(np.arange(0, 180, 20), np.arange(0, 180, 20))

# plt.legend()

plot_title = "Darkness detection accuracy all black plus negatives"
# plt.title(plot_title)
plt.xlabel("Average pixel value of frame (blue channel)")
plt.ylabel("Detection Accuracy")

print(p)
if save:
    save_fig(plots_save_folder, plot_title, fig=None)

In [ ]:
plt.scatter(orig_x_pruned, orig_y_pruned, 
            s=orig_weights_pruned/2, alpha=.4, label='Original validation', c='b')

plt.figure()
plt.scatter(orig_x, orig_y, 
            s=orig_weights/2, alpha=.4, label='Original validation', c='b')

In [ ]:
# residuals = []
# for x, y, w in zip(x_pruned, y_pruned, weights_pruned):
#     y_pred = piecewise_linear(x, *p)
#     y_pred = .93
#     if y - y_pred > -1:
#         for _ in range(int(w)):
#             residuals.append(y - y_pred)
# plt.hist(residuals, bins = 50)
residuals = []
for x_val, y_val, w in zip(x, y, weights):
#     w=1
    y_pred = piecewise_linear(x_val, *p)
#     y_pred = .94
    residual = np.abs(y_pred - y_val)
#     w = 1
    for _ in range(int(w)):
        residuals.append(residual)
_ = plt.hist(residuals, bins = 100)
print(f"mean: {np.mean(residuals)}")

In [ ]:
break_point = 0


break_y = orig_y[orig_x > break_point]
break_x = orig_x[orig_x > break_point]
break_weights = orig_weights[orig_x > break_point]

In [ ]:
mean = np.sum(break_y * break_weights / np.sum(break_weights))
print(f"mean: {mean}")
print(f"median: {np.median(break_y)}")

std_numerator = np.sum((break_y - mean) ** 2 * break_weights)
std_denomenator = ((len(break_weights) - 1) / len(break_weights)) * np.sum(break_weights)
std = np.sqrt(std_numerator / std_denomenator)
print(f"std: {std}")

In [ ]:
len(break_y)

In [ ]:
print(f" 25: {np.percentile(break_y, 25)}, 75: {np.percentile(break_y, 75)}")
weighted_y = np.repeat(break_y, break_weights.astype(int))
print(f" 25: {np.percentile(weighted_y, 25)}, 50: {np.percentile(weighted_y, 50)}, 75: {np.percentile(weighted_y, 75)}")

# old_method
old_orig_x = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'darkness_mean'].values
old_orig_y = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'old_method_fraction_total'].values
old_orig_weights = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'total_bats'].values

print(f" 25: {np.percentile(old_orig_y, 25)}, 75: {np.percentile(old_orig_y, 75)}")
old_weighted_y = np.repeat(old_orig_y, old_orig_weights.astype(int))
print(f" 25: {np.percentile(old_weighted_y, 25)}, 50: {np.percentile(old_weighted_y, 50)}, 75: {np.percentile(old_weighted_y, 75)}")


In [ ]:
x = val_df['darkness-mean'].values
y = val_df['new-method-fraction-total'].values
weights = val_df['total-bats'].values

y_old = val_df['old-method-fraction-total'].values

In [ ]:
print(f" 25: {np.percentile(y, 25)}, 75: {np.percentile(y, 75)}")
weighted_y = np.repeat(y, weights.astype(int))
print(f" 25: {np.percentile(weighted_y, 25)}, 50: {np.percentile(weighted_y, 50)}, 75: {np.percentile(weighted_y, 75)}")

# old_method
print(f" 25: {np.percentile(y_old, 25)}, 75: {np.percentile(y_old, 75)}")
old_weighted_y = np.repeat(y_old, weights.astype(int))
print(f" 25: {np.percentile(y_old, 25)}, 50: {np.percentile(y_old, 50)}, 75: {np.percentile(y_old, 75)}")


In [ ]:
mean = np.sum(y * weights / np.sum(weights))
print(f"mean: {mean}")

mean = np.sum(y_old * weights / np.sum(weights))
print(f"mean: {mean}")

In [ ]:
plt.scatter(x, y)
plt.scatter(x, y_old)
plt.ylim(0,2)

print()

In [ ]:
plt.hist(break_y, bins=20)
plt.figure()
plt.hist(old_orig_y, bins = 20)

In [ ]:
val_df['camera-type'] = "new"
val_df.loc[val_df['camera-name']=='FibweParking', 'camera-type'] = 'gp4'
val_df.loc[val_df['camera-name']=='FibweParking2', 'camera-type'] = 'gp4'
val_df.loc[val_df['camera-name']=='FibwePublic', 'camera-type'] = 'gp4'
val_df.loc[val_df['camera-name']=='MusoleTower', 'camera-type'] = 'gp4'
val_df.loc[val_df['camera-name']=='MusolePath', 'camera-type'] = 'gp4'
val_df.loc[val_df['camera-name']=='MusolePath2', 'camera-type'] = 'gp4'


orig_df['camera-type'] = "new"
orig_df.loc[orig_df['clip-num']=='FibweParking', 'camera-type'] = 'gp4'
orig_df.loc[orig_df['clip-num']=='FibweParking2', 'camera-type'] = 'gp4'
orig_df.loc[orig_df['clip-num']=='FibwePublic', 'camera-type'] = 'gp4'
orig_df.loc[orig_df['clip-num']=='MusoleTower', 'camera-type'] = 'gp4'
orig_df.loc[orig_df['clip-num']=='MusolePath', 'camera-type'] = 'gp4'
orig_df.loc[orig_df['clip-num']=='MusolePath2', 'camera-type'] = 'gp4'

In [ ]:
orig_df.head()

In [ ]:
orig_df['clip-num'].unique()

In [ ]:
camera_types = ['gp4', 'new']


save = False

for camera_type in camera_types:
    select_rows = ((val_df['new-method-fraction-total'] > 0)
                   & (val_df['camera-type'] == camera_type))
    x_pruned = val_df['darkness-mean'][select_rows].values
    y_pruned = val_df['new-method-fraction-total'][select_rows].values
    weights_pruned = val_df['total-bats'][select_rows].values

    select_rows = ((orig_df['new_method_fraction_total'] > 0)
                   & (orig_df['camera-type'] == camera_type))
    orig_x_pruned = orig_df['darkness_mean'][select_rows].values
    orig_y_pruned = orig_df['new_method_fraction_total'][select_rows].values
    orig_weights_pruned = orig_df['total_bats'][select_rows].values

    x = val_df['darkness-mean'][val_df['camera-type'] == camera_type].values
    y = val_df['new-method-fraction-total'][val_df['camera-type'] == camera_type].values
    weights = val_df['total-bats'][val_df['camera-type'] == camera_type].values

    
    select_orig_rows = (~np.isnan(orig_df['total_bats'])
                        &(orig_df['total_bats'] > 0)
                        &(orig_df['camera-type'] == camera_type))
    orig_x = orig_df.loc[select_orig_rows, 'darkness_mean'].values
    orig_y = orig_df.loc[select_orig_rows, 'new_method_fraction_total'].values
    orig_weights = orig_df.loc[select_orig_rows, 'total_bats'].values

    x = np.hstack([x, orig_x])
    y = np.hstack([y, orig_y])
    weights = np.hstack([weights, orig_weights])

    sigmas = 1 / weights

    def piecewise_linear(x, x0, y0, k1, k2):
        return np.piecewise(x, [x < x0], [lambda x:k1*x + y0-k1*x0, lambda x:k2*x + y0-k2*x0])


    xd = np.linspace(5, 140, 140)
    x_pruned = np.hstack([x_pruned, orig_x_pruned])
    y_pruned = np.hstack([y_pruned, orig_y_pruned])
    weights_pruned = np.hstack([weights_pruned, orig_weights_pruned])
    
#     plt.figure()
    
    plt.scatter(x_pruned, y_pruned, 
                s=weights_pruned, alpha=.4)
    p , e = optimize.curve_fit(piecewise_linear, x, y, p0=[20, 1, 1, -1], sigma=sigmas)
    
    residuals = y- piecewise_linear(x, *p)
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y-np.mean(y))**2)
    r_squared = 1 - (ss_res / ss_tot)
    print(f"{camera_type}: r_squared: {r_squared}")
    
    if camera_type == "gp4":
        label = "GoPro 4"
    else:
        label = "GoPro 6 or 7"
    
    plt.plot(xd, piecewise_linear(xd, *p), label=label)
    plot_title = "Brightness detection accuracy new and old cameas"
    plt.title(plot_title)
    plt.xlabel("Average pixel value of frame (blue channel)")
    plt.ylabel("Detection Accuracy")
#     print(p)
#     perr = np.sqrt(np.diag(e))
#     print(perr)
if save:
    plt.legend()
    save_fig(plots_save_folder, plot_title, fig=None)

In [ ]:
val_df['date-folder']

In [ ]:
dates = ['16', '17']


save = True

for date in dates:
    orig_date = date + ' Nov'
    val_date = date + 'Nov'
    select_rows = ((val_df['new-method-fraction-total'] > 0)
                   & (val_df['date-folder'] == val_date))
    x_pruned = val_df['darkness-mean'][select_rows].values
    y_pruned = val_df['new-method-fraction-total'][select_rows].values
    weights_pruned = val_df['total-bats'][select_rows].values

    select_rows = ((orig_df['new_method_fraction_total'] > 0)
                   & (orig_df['date_folder'] == orig_date))
    orig_x_pruned = orig_df['darkness_mean'][select_rows].values
    orig_y_pruned = orig_df['new_method_fraction_total'][select_rows].values
    orig_weights_pruned = orig_df['total_bats'][select_rows].values

    x = val_df['darkness-mean'][val_df['date-folder'] == val_date].values
    y = val_df['new-method-fraction-total'][val_df['date-folder'] == val_date].values
    weights = val_df['total-bats'][val_df['date-folder'] == val_date].values

    
    select_orig_rows = (~np.isnan(orig_df['total_bats'])
                        &(orig_df['total_bats'] > 0)
                        &(orig_df['date_folder'] == orig_date))
    orig_x = orig_df.loc[select_orig_rows, 'darkness_mean'].values
    orig_y = orig_df.loc[select_orig_rows, 'new_method_fraction_total'].values
    orig_weights = orig_df.loc[select_orig_rows, 'total_bats'].values

    x = np.hstack([x, orig_x])
    y = np.hstack([y, orig_y])
    weights = np.hstack([weights, orig_weights])
    
    
    
    print(date, 'acc:', np.sum(y * (weights / np.sum(weights))))

    sigmas = 1 / weights

    def piecewise_linear(x, x0, y0, k1, k2):
        return np.piecewise(x, [x < x0], [lambda x:k1*x + y0-k1*x0, lambda x:k2*x + y0-k2*x0])


    xd = np.linspace(5, 140, 140)
    x_pruned = np.hstack([x_pruned, orig_x_pruned])
    y_pruned = np.hstack([y_pruned, orig_y_pruned])
    weights_pruned = np.hstack([weights_pruned, orig_weights_pruned])
    
    plt.scatter(x_pruned, y_pruned, 
                s=weights_pruned, alpha=.4, label=(date + 'Nov'))
    p , e = optimize.curve_fit(piecewise_linear, x, y, p0=[20, 1, 1, -1], sigma=sigmas)
    
    residuals = y- piecewise_linear(x, *p)
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y-np.mean(y))**2)
    r_squared = 1 - (ss_res / ss_tot)
    print(f"{date}: r_squared: {r_squared}")
    
    
    plt.plot(xd, piecewise_linear(xd, *p))
    plot_title = "Detection accuracy 16th vs 17th"
    plt.title(plot_title)
    plt.xlabel("Average pixel value of frame (blue channel)")
    plt.ylabel("Detection Accuracy")
    print(p)
    perr = np.sqrt(np.diag(e))
    print(perr)
plt.legend()
if save:
    
    save_fig(plots_save_folder, plot_title, fig=None)

In [ ]:
x_pruned = val_df['darkness-mean'][val_df['new-method-fraction-total']>0].values
y_pruned = val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0].values
weights_pruned = val_df['total-bats'][val_df['new-method-fraction-total']>0].values

orig_x_pruned = orig_df['darkness_mean'][orig_df['new_method_fraction_total']>0].values
orig_y_pruned = orig_df['new_method_fraction_total'][orig_df['new_method_fraction_total']>0].values
orig_weights_pruned = orig_df['total_bats'][orig_df['new_method_fraction_total']>0].values

x = val_df['darkness-mean'].values
y = val_df['new-method-fraction-total'].values
weights = val_df['total-bats'].values

orig_x = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'darkness_mean'].values
orig_y = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'new_method_fraction_total'].values
orig_weights = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'total_bats'].values 

x = np.hstack([x, orig_x])
y = np.hstack([y, orig_y])
weights = np.hstack([weights, orig_weights])

sigmas = 1 / weights

def piecewise_linear(x, x0, y0, k1, k2):
    return np.piecewise(x, [x < x0], [lambda x:k1*x + y0-k1*x0, lambda x:k2*x + y0-k2*x0])


xd = np.linspace(0, 170, 170)
x_pruned = np.hstack([x_pruned, orig_x_pruned])
y_pruned = np.hstack([y_pruned, orig_y_pruned])
weights_pruned = np.hstack([weights_pruned, orig_weights_pruned])
# Raw
# plt.scatter(x, y, 
#             s=weights, alpha=.4, label='Falloff validation')

# Pruned
plt.scatter(x_pruned, y_pruned, 
            s=weights_pruned, alpha=.4, label='Falloff validation', c='k')

# Center points
# plt.scatter(x_pruned, y_pruned, 
#             s=1, alpha=.4, label='Falloff validation', c='w')

# Just original validation
# plt.scatter(orig_x_pruned, orig_y_pruned, 
#             s=orig_weights_pruned/2, alpha=.4, label='Original validation', c='b')

p , e = optimize.curve_fit(piecewise_linear, x, y, p0=[10, 1, 1, 1], sigma=sigmas)
plt.plot(xd, piecewise_linear(xd, *p), c='gray')
plt.margins(x=0)
plt.xticks(np.arange(0, 180, 20), np.arange(0, 180, 20))

# plt.legend()

plot_title = "Darkness detection accuracy all black"
# plt.title(plot_title)
plt.xlabel("Average pixel value of frame (blue channel)")
plt.ylabel("Detection Accuracy")

# print(p)
# save_fig(plots_save_folder, plot_title, fig=None)

In [ ]:
plots_save_folder

In [ ]:
orig_x = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'darkness_mean'].values
orig_y = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'total_bats_new_method'].values
orig_weights = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'total_bats'].values 

x = np.any(np.isnan(orig_x))
y =  np.where(np.isnan(orig_y))
weights =  np.any(np.isnan(orig_weights))
print(x, y, weights)

In [ ]:
p[1]-p[3]*p[0], p[1]-p[2]*p[0]

In [ ]:
x_pruned = val_df['darkness-mean'][val_df['new-method-fraction-total']>0].values
y_pruned = val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0].values
weights_pruned = val_df['total-bats'][val_df['new-method-fraction-total']>0].values

orig_x_pruned = orig_df['darkness_mean'][orig_df['new_method_fraction_total']>0].values
orig_y_pruned = orig_df['new_method_fraction_total'][orig_df['new_method_fraction_total']>0].values
orig_weights_pruned = orig_df['total_bats'][orig_df['new_method_fraction_total']>0].values

x = val_df['darkness-mean'].values
y = val_df['new-method-fraction-total'].values
weights = val_df['total-bats'].values

orig_x = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'darkness_mean'].values
orig_y = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'new_method_fraction_total'].values
orig_weights = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'total_bats'].values 

x = np.hstack([x, orig_x])
y = np.hstack([y, orig_y])
weights = np.hstack([weights, orig_weights])

sigmas = 1 / weights

def piecewise_linear(x, x0, y0, k1, k2):
    return np.piecewise(x, [x < x0], [lambda x:k1*x + y0-k1*x0, lambda x:k2*x + y0-k2*x0])


xd = np.linspace(0, 160, 160)
x_pruned = np.hstack([x_pruned, orig_x_pruned])
y_pruned = np.hstack([y_pruned, orig_y_pruned])
weights_pruned = np.hstack([weights_pruned, orig_weights_pruned/2])
plt.scatter(x_pruned, y_pruned, 
            s=weights_pruned, alpha=.4, label='Deep learning')

p , e = optimize.curve_fit(piecewise_linear, x, y, p0=[30, 1, 1, 1], sigma=sigmas)
plt.plot(xd, piecewise_linear(xd, *p))
plt.legend()


x_pruned = val_df['darkness-mean'][val_df['old-method-fraction-total']>0].values
y_pruned = val_df['old-method-fraction-total'][val_df['old-method-fraction-total']>0].values
weights_pruned = val_df['total-bats'][val_df['old-method-fraction-total']>0].values
x_pruned = x_pruned[y_pruned<2]
weights_pruned = weights_pruned[y_pruned<2]
y_pruned = y_pruned[y_pruned<2]


orig_x_pruned = orig_df['darkness_mean'][orig_df['old_method_fraction_total']>0].values
orig_y_pruned = orig_df['old_method_fraction_total'][orig_df['old_method_fraction_total']>0].values
orig_weights_pruned = orig_df['total_bats'][orig_df['old_method_fraction_total']>0].values

x = val_df['darkness-mean'].values
y = val_df['old-method-fraction-total'].values
weights = val_df['total-bats'].values

orig_x = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'darkness_mean'].values
orig_y = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'old_method_fraction_total'].values
orig_weights = orig_df.loc[(~np.isnan(orig_df['total_bats'])&(orig_df['total_bats']>0)), 'total_bats'].values

x = np.hstack([x, orig_x])
y = np.hstack([y, orig_y])
weights = np.hstack([weights, orig_weights/2])

sigmas = 1 / weights

def piecewise_linear(x, x0, y0, k1, k2):
    return np.piecewise(x, [x < x0], [lambda x:k1*x + y0-k1*x0, lambda x:k2*x + y0-k2*x0])


xd = np.linspace(0, 160, 160)
x_pruned = np.hstack([x_pruned, orig_x_pruned])
y_pruned = np.hstack([y_pruned, orig_y_pruned])
weights_pruned = np.hstack([weights_pruned, orig_weights_pruned/2])
plt.scatter(x_pruned, y_pruned, 
            s=weights_pruned, alpha=.4, label="Old method")
p , e = optimize.curve_fit(piecewise_linear, x, y, p0=[20, 1, 1, 1], sigma=sigmas)
plt.plot(xd, piecewise_linear(xd, *p))
plt.legend()
print(p)

plot_title = "Darkness detection accuracy comparison"
plt.title(plot_title)
plt.xlabel("Average pixel value of frame (blue channel)")
plt.ylabel("Detection Accuracy")

save_fig(plots_save_folder, plot_title, fig=None)

In [ ]:
print(f'function 1: y = {p[2]}x + {p[1] - p[2]*p[0]}')
print(f'function 2: y = {p[3]}x + {p[1] - p[3]*p[0]}')

In [ ]:
print(f'function 1: y = {p[2]}x + {p[1] - p[2]*p[0]}')
print(f'function 2: y = {p[3]}x + {p[1] - p[3]*p[0]}')

In [ ]:
plots_save_folder = '.../bats-data/plots/falloff-validation-supplement'

os.makedirs(plots_save_folder, exist_ok=True)

observation_groups = val_df.loc[~np.isnan(val_df['total-bats'])].groupby(['date-folder', 'camera-name'])

percentiles = [.65, .8, .95, .98]

old_frac = []
new_frac = []

for key,item in observation_groups:
    a_group = observation_groups.get_group(key)
    a_group = a_group.sort_values('clip-num').reset_index()
    fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(10, 4))
    first_frames = [name.split('-')[6] for name in a_group['video-clip-name']] 
    ax1.plot(first_frames, a_group['total-bats'], label='human')
    ax1.plot(first_frames, a_group['total-bats-new-method'], label='software')
    ax1.set_ylim(0, np.max(a_group['total-bats'] + a_group['total-bats-new-method']))
    ax1.set_xlabel('Frame number')
    ax1.set_ylabel('Bats in clip')
    ax1.legend()
    cumsum_total = np.cumsum(a_group['total-bats'])
    cumsum_new = np.cumsum(a_group['total-bats-new-method'])
    ax2.plot(first_frames, cumsum_total, label='human')
    ax2.plot(first_frames, cumsum_new, label='software')
    ax2.set_xlabel('Frame number')
    ax2.set_ylabel('Bat accumulation across clips')
    frac_seen_new = cumsum_new.iloc[-1] / cumsum_total.iloc[-1]
    frac_seen_old = cumsum_old.iloc[-1] / cumsum_total.iloc[-1]
    old_frac.append(frac_seen_old)
    new_frac.append(frac_seen_new)
    camera = a_group['camera-name'].iloc[0]
    date = a_group['date-folder'].iloc[0]
    fig.suptitle(f"{date}, {camera}", fontsize=14)
    ax2.legend()
    
    plot_title = f"error falloff and accumulation {date} {camera}"

    save_fig(plots_save_folder, plot_title, fig=fig)

In [ ]:
plt.style.use('default')

In [ ]:
title = 'total bats vs estimated bats'
plt.figure(figsize=(5, 5))
plt.scatter(val_df['total-bats'], val_df['total-bats-new-method'])
plt.title(title)
plt.gca().set_aspect('equal', adjustable='box')
plt.plot([0, 900], [0, 900], c='r', linewidth=1)
plt.xlabel('total bats')
plt.ylabel('counted bats')
save_fig(plots_save_folder, title)

In [ ]:
plt.figure(figsize=(5,5))
title = 'frame darkness vs fraction bats detected'
plt.scatter(val_df['darkness-mean'][val_df['new-method-fraction-total']>0], 
            val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0], 
             label='mean-darkness')
plt.scatter(val_df['darkness-median'][val_df['new-method-fraction-total']>0], 
            val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0], 
             label='median-darkness')
plt.title(title)
plt.xlabel('pixel value')
plt.ylabel('fraction total bats counted')
# plt.scatter(val_df['darkness-max'], val_df['new-method-fraction-total'], s=2, label='max-light')
plt.legend()
save_fig(plots_save_folder, title)

In [ ]:
plt.figure(figsize=(5,5))
title = 'mean darkness vs fraction bats detected'
plt.scatter(val_df['darkness-mean'][val_df['new-method-fraction-total']>0], 
            val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0], 
             label='mean-darkness')
plt.title(title)
plt.xlabel('pixel value')
plt.ylabel('fraction total bats counted')
# plt.scatter(val_df['darkness-max'], val_df['new-method-fraction-total'], s=2, label='max-light')
plt.legend()
save_fig(plots_save_folder, title)

In [ ]:
title = 'frame darkness vs fraction detected, circle size is total bats'
plt.figure(figsize=(5,5))
plt.scatter(val_df['darkness-mean'][val_df['new-method-fraction-total']>0], 
            val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0], 
            s=val_df['total-bats'][val_df['new-method-fraction-total']>0], 
            label='total bats',
            alpha=.3)
plt.title(title)
plt.legend()
plt.xlabel('mean frame darkness')
plt.ylabel('fraction total bats counted')
save_fig(plots_save_folder, title)

In [ ]:
plt.figure()
plt.scatter(val_df['darkness-mean'][val_df['new-method-fraction-total']>0], 
            val_df['old-method-fraction-total'][val_df['new-method-fraction-total']>0], 
            s=val_df['total-bats'][val_df['new-method-fraction-total']>0], 
            label='old')
plt.legend()
plt.title('frame darkness vs fraction detected, circle size is total bats')

In [ ]:
from numpy.polynomial import Polynomial

In [ ]:
p_all = Polynomial.fit(val_df['darkness-mean'][val_df['new-method-fraction-total']>0],
                   val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0],
                   w=val_df['total-bats'][val_df['new-method-fraction-total']>0],
                   deg=4)

p_16 = Polynomial.fit(val_df['darkness-mean'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='16Nov'],
                   val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='16Nov'],
                   w=val_df['total-bats'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='16Nov'],
                   deg=4)

p_17 = Polynomial.fit(val_df['darkness-mean'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='17Nov'],
                   val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='17Nov'],
                   w=val_df['total-bats'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='17Nov'],
                   deg=4)

In [ ]:
title = 'frame darkness vs fraction detected, with poly fit'
x = np.arange(np.max(val_df['darkness-mean'][val_df['new-method-fraction-total']>0].values))
y_all = p_all(x)
plt.plot(x, y_all, label='4th degree poly fit')

plt.scatter(val_df['darkness-mean'][val_df['new-method-fraction-total']>0],
            val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0],
            s=val_df['total-bats'][val_df['new-method-fraction-total']>0], 
            alpha=.3)

plt.legend()
plt.title(title)
plt.xlabel('mean frame darkness')
plt.ylabel('fraction total bats counted')
save_fig(plots_save_folder, title)

In [ ]:
title = 'frame darkness vs fraction detected, with fraction counted'
frac_passed = [.65,.8, .95, .98]

x = np.arange(np.max(val_df['darkness-mean'][val_df['new-method-fraction-total']>0].values))
y_all = p_all(x)
plt.plot(x, y_all, label='4th degree poly fit')

for clip_num in range(4):
    plt.scatter(val_df['darkness-mean'][val_df['new-method-fraction-total']>0][val_df['clip-num'].astype(int)==clip_num],
                val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0][val_df['clip-num'].astype(int)==clip_num],
                s=val_df['total-bats'][val_df['new-method-fraction-total']>0][val_df['clip-num'].astype(int)==clip_num], 
                alpha=.3,
                label=f'{frac_passed[clip_num]} passed')

plt.legend()
plt.title(title)
plt.xlabel('mean frame darkness')
plt.ylabel('fraction total bats counted')
save_fig(plots_save_folder, title)

In [ ]:
title = 'frame darkness vs fraction detected, with date'
plt.scatter(val_df['darkness-mean'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='17Nov'],
            val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='17Nov'],
            s=val_df['total-bats'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='17Nov'], 
            alpha=.3, label='17Nov')
plt.scatter(val_df['darkness-mean'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='16Nov'],
            val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='16Nov'],
            s=val_df['total-bats'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='16Nov'], 
            alpha=.3, label='16Nov')
plt.legend()
plt.title(title)
plt.xlabel('mean frame darkness')
plt.ylabel('fraction total bats counted')
save_fig(plots_save_folder, title)

In [ ]:
title = 'frame darkness vs fraction detected, with date and fit'
x = np.arange(np.max(val_df['darkness-mean'][val_df['new-method-fraction-total']>0].values))
y_all = p_all(x)
plt.plot(x, y_all, label='combined')
y_16 = p_16(x)
plt.plot(x, y_16, label='16Nov')
y_17 = p_17(x)
plt.plot(x, y_17, label='17Nov')
plt.scatter(val_df['darkness-mean'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='17Nov'],
            val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='17Nov'],
            s=val_df['total-bats'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='17Nov'], 
            alpha=.3)
plt.scatter(val_df['darkness-mean'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='16Nov'],
            val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='16Nov'],
            s=val_df['total-bats'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='16Nov'], 
            alpha=.3)
plt.legend()
plt.title(title)
plt.xlabel('mean frame darkness')
plt.ylabel('fraction total bats counted')
save_fig(plots_save_folder, title)

In [ ]:
clip_num_date_camera = val_df[val_df['new-method-fraction-total']>0].groupby(['date-folder', 'camera-name'])

In [ ]:
date='17Nov'

x = [.65, .80, .95, .98]
fig, axs = plt.subplots(5, 2, figsize=(10,10))
for ind, name in enumerate(val_df[val_df['date-folder']=='16Nov']['camera-name'].unique()):
    plt.figure()
    axs[ind%5, ind//5].plot(x, val_df[val_df['camera-name']==name][val_df['date-folder']=='16Nov']['total-bats'], label='total')
    axs[ind%5, ind//5].plot(x, val_df[val_df['camera-name']==name][val_df['date-folder']=='16Nov']['total-bats-new-method'], label='counted')
    axs[ind%5, ind//5].legend()
fig.suptitle(date, fontsize=14)
save_fig(plots_save_folder, '17Nov', fig)

In [ ]:
val_df['camera-name']

In [ ]:
clip_num_group = val_df[val_df['new-method-fraction-total']>0].groupby('clip-num')

In [ ]:
clip_num_date_group = val_df[val_df['new-method-fraction-total']>0].groupby(['date-folder', 'clip-num'])
print(clip_num_date_group.sum()['total-bats-new-method'])
print(clip_num_date_group.sum()['total-bats'])
print(clip_num_date_group.sum()['total-bats-new-method']['16Nov'].values / clip_num_date_group.sum()['total-bats']['16Nov'].values)
print(clip_num_date_group.sum()['total-bats-new-method']['17Nov'].values / clip_num_date_group.sum()['total-bats']['17Nov'].values)

In [ ]:
nov16_color = ('#3498DB')
nov17_color = ('#E74C3C')

plt.figure()
plt.scatter(val_df['clip-num'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='17Nov'], 
            val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='17Nov'], 
            s=val_df['total-bats'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='17Nov'],
           alpha=.3, color=nov16_color)

plt.scatter(val_df['clip-num'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='16Nov'], 
            val_df['new-method-fraction-total'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='16Nov'], 
            s=val_df['total-bats'][val_df['new-method-fraction-total']>0][val_df['date-folder']=='16Nov'],
           alpha=.3, color=nov17_color)

weighted_average = (clip_num_group.sum()['total-bats-new-method'].values 
                    / clip_num_group.sum()['total-bats'].values
                   )
plt.scatter(np.arange(4), weighted_average, marker= '_', c='g', label='both days')
weighted_average_16 = (clip_num_date_group.sum()['total-bats-new-method']['16Nov'].values 
                       / clip_num_date_group.sum()['total-bats']['16Nov'].values)
plt.scatter(np.arange(4), weighted_average_16, marker= '_', c=nov16_color, label='16 Nov')
weighted_average_17 = (clip_num_date_group.sum()['total-bats-new-method']['17Nov'].values 
                       / clip_num_date_group.sum()['total-bats']['17Nov'].values)
plt.scatter(np.arange(4), weighted_average_17, marker= '_', c=nov17_color, label='17 Nov')
plt.title('fraction of bats detected at .65, .80, .95, .98 of total bats')
plt.legend()

In [ ]:
plt.scatter(val_df['total-bats'], val_df['total-bats-new-method'])
plt.title('total bats vs. estimated bats')
plt.figure()
plt.scatter(val_df['darkness-mean'], val_df['new-method-fraction-total'], s=2, label='mean-darkness')
plt.scatter(val_df['darkness-median'], val_df['new-method-fraction-total'], s=2, label='median-darkness')
plt.title('frame darkness vs fraction detected')
# plt.scatter(val_df['darkness-max'], val_df['new-method-fraction-total'], s=2, label='max-light')
plt.legend()
plt.figure()
plt.scatter(val_df['darkness-mean'], val_df['new-method-fraction-total'], s=val_df['total-bats'], label='new')
plt.scatter(val_df['darkness-mean'], val_df['old-method-fraction-total'], s=val_df['total-bats'], label='old')
plt.legend()
plt.title('frame darkness vs fraction detected, circle size is total bats')

In [ ]:
val_df.head()

In [ ]:
day_folders = sorted(glob.glob('.../kasanka-bats/processed/deep-learning/*Nov'))
observations = {}
for day_folder in day_folders[:2]:
    print(day_folder)

    date = os.path.basename(day_folder)
    track_files = sorted(glob.glob(os.path.join(day_folder, '*/crossing_tracks.npy')))
    observations[date] = {}
    for track_file in track_files: 
        camera = track_file.split('/')[-2]
        obs = {'date': date,
               'camera': camera,
               }
        crossing_tracks_list = np.load(track_file, allow_pickle=True)
        passing_bat_frame = []
        passing_bat_size = []
        passing_bat_track_ind = []
        passing_bat_direction = []
        passing_bat_track_id = []
        for track_ind, track in enumerate(crossing_tracks_list):
            if track['crossed'] > 0:
                passing_bat_frame.append(track['crossed'])
                passing_bat_size.append(track['mean_wing'])
                passing_bat_track_id.append(track_ind)
                passing_bat_direction.append(1)

            elif track['crossed'] < 0:
                passing_bat_frame.append(track['crossed'])
                passing_bat_size.append(track['mean_wing'])
                passing_bat_track_id.append(track_ind)
                passing_bat_direction.append(-1)

        obs['frames'] = np.array(passing_bat_frame)
        obs['mean_wing'] = np.array(passing_bat_size)
        obs['ids'] = np.array(passing_bat_track_id)
        obs['direction'] = np.array(passing_bat_direction)
        observations[date][camera] = obs

In [ ]:
for date, day_obs in observations.items():
    for cam_ind, (cam_name, obs) in enumerate(day_obs.items()):
        bat_accumulation = get_bat_accumulation(obs['frames'])
        observations[date][cam_name]['accumulation'] = bat_accumulation


In [ ]:


percentiles = [.65, .8, .95, .98]
accumulation = observations['16Nov']['Puku']['accumulation']
accumulation_percent = accumulation / accumulation[-1]
plt.plot(accumulation_percent)
for percentile in percentiles:
    obs_ind = np.argmax(accumulation_percent > percentile)
    plt.scatter(obs_ind, accumulation_percent[obs_ind])

In [ ]:
np.argmax(accumulation_percent > percentile)

In [ ]:
import glob
import os
parent_video_folder = '.../kasanka-bats/gopros/' 

all_video_files = []
for date in ['16Nov', '17Nov', '18Nov', '19Nov', '20Nov']:
    all_video_files.extend(glob.glob(parent_video_folder + date + '/*/*.MP4'))
#     video_files.extend(glob.glob(parent_video_folder + date + '/BBC/*.MP4'))
all_video_files.sort()
    # vf = glob.glob(parent_video_folder + '/14_11_g/*.MP4')
    # vf.sort()
    # video_files.extend(vf)
# print(*video_files, sep='\n')

median_files = 0
mean_files = 0
max_files = 0

video_files = []

for video_file in all_video_files:
    
    video_name = os.path.basename(video_file)
    video_name = video_name.split('.')[-2]
    parent_output_folder = '/home/golden/kenya_drones/kasanka-bats/processed/deep-learning'
    observation_name = video_file.split('/')[-2]
    output_folder = os.path.join(parent_output_folder, video_file.split('/')[-3], observation_name)
    
    if os.path.exists(os.path.join(output_folder, f'mean-blue-{video_name}.npy')):
        mean_files += 1
    else:
        video_files.append(video_file)
    if os.path.exists(os.path.join(output_folder, f'median-blue-{video_name}.npy')):
        median_files += 1
    if os.path.exists(os.path.join(output_folder, f'max-blue-{video_name}.npy')):
        max_files += 1

print(f'num mean: {mean_files}, num median: {mean_files}, num max: {max_files}')
print(f'fraction mean: {mean_files / len(all_video_files)}')

In [ ]:
len(all_video_files)